<a href="https://colab.research.google.com/github/hannape/IBAC-Biodiv/blob/master/03_CNN_fit_and_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fitujemy model i robimy predykcję na 20 nagraniach. 

Na podstawie I: Copy of CNN-weighting-IBAC.ipynb

In [0]:
# previous version, with hidden code and comments - cnn_scorer_not_working, cnn_weighting

from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function


In [0]:
import keras
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.utils import class_weight
from sklearn.metrics import  make_scorer
from sklearn.metrics import log_loss
K.set_image_dim_ordering('th')
import numpy as np
from sklearn import preprocessing
import glob
import os
import random
import functools
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler, CSVLogger
import math
import datetime
import tensorflow as tf
import functools
from functools import partial, update_wrapper


from keras.models import load_model


In [0]:
from numpy.random import seed
seed(667)
from tensorflow import set_random_seed
set_random_seed(667)
import random
random.seed()

'''
#################### Rep 1 - spektro ####################
# rep 1  ------- 63 x 148 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep1/X_train_rep1.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep1/y_train.npy', allow_pickle=True)
'''
# rep 1V2 ------- 63 x 148 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep1V2/X_train_rep1.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep1V2/y_train_rep1.npy', allow_pickle=True)
'''
# rep 1b ------- 63 x 63 ------
X_train1 = numpy.load('drive/My Drive/rep1b/X_train_rep1b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep1b/y_train_rep1b.npy', allow_pickle=True)

#################### Rep 3 - mel-spektro ####################

# rep 3 ------- 60 x 111 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep3/X_train_rep3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep3/y_train.npy', allow_pickle=True)

# rep 3V2 ------- 60 x 111 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep3V2/X_train_rep3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep3V2/y_train_rep3.npy', allow_pickle=True)

# rep 3b ------- 60 x 63 ------
X_train1 = numpy.load('drive/My Drive/rep3b/X_train_rep3b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep3b/y_train_rep3b.npy', allow_pickle=True)

# rep 3V3 ------- 60 x 148 ------
X_train1 = numpy.load('drive/My Drive/rep3V3/X_train_rep3V3.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep3V3/y_train_rep3V3.npy', allow_pickle=True)

#################### Rep 5 - mel-spektro ####################

# rep 5 ------- 64 x 61 ------ TRAIN 17.3k
X_train1 = numpy.load('drive/My Drive/rep5/X_train_rep5.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep5/y_train.npy', allow_pickle=True)

# rep 5V2 ------- 64 x 61 ------ TRAIN 15.9
X_train1 = numpy.load('drive/My Drive/rep5V2/X_train_rep5.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep5V2/y_train_rep5.npy', allow_pickle=True)

# rep 5b ------- 64 x 149 ------
X_train1 = numpy.load('drive/My Drive/rep5b/X_train_rep5b.npy', allow_pickle=True) # 1 and 0 in order
y_train1 = numpy.load('drive/My Drive/rep5b/y_train_rep5b.npy', allow_pickle=True)
'''

print('Training set size:')
print(np.shape(X_train1))
print(np.shape(y_train1))


In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.2, random_state=667) # default shuffle:true
r = np.shape(X_val)[1]
s = np.shape(X_val)[2]

del X_train1, y_train1

exemp = 4# exemplary frame
plt.subplot(1, 2, 1)
plt.imshow(X_train[exemp], cmap="jet")
plt.subplot(1, 2, 2)
plt.imshow(X_val[exemp], cmap="jet")

print(y_train[exemp],y_val[exemp])

X_train = X_train.reshape(X_train.shape[0], 1, r, s).astype('float32')
X_val = X_val.reshape(X_val.shape[0], 1, r, s).astype('float32')

print('Training set size:')
print(np.shape(X_train))
print(np.shape(y_train))

print('Validation set size:')
print(np.shape(X_val))
print(np.shape(y_val))

In [0]:
# https://github.com/keras-team/keras/issues/2115

### definiowanie wag 
for_zeros = 0.1
for_ones = 0.9
###

### SCORERS

def binary_crossentropy_weigted(y_true, y_pred, class_weights):
	y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
  
	loss = K.mean(class_weights*(-y_true * K.log(y_pred) - (1.0 - y_true) * K.log(1.0 - y_pred)),axis=-1)
	return loss

def weighted_binary_crossentropy( y_true, y_pred, weights_10) :
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weights_10[0] + (1 - y_true) * K.log(1 - y_pred) * weights_10[1])
    return K.mean(logloss, axis=-1)

custom_loss1 = partial(binary_crossentropy_weigted, class_weights=np.array([for_zeros,for_ones])) ## scoring for model.compile
custom_loss1.__name__ ='binary_crossentropy_weigted'

custom_loss2 = partial(weighted_binary_crossentropy, weights_10=np.array([for_ones,for_zeros])) ## scoring for model.compile
custom_loss2.__name__ ='weighted_binary_crossentropy'

## AUC METRIC
def as_keras_metric(method):
    import functools
    from keras import backend as K
    
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper
  
auc_roc = as_keras_metric(tf.metrics.auc)



In [0]:
######### Fitowanie modelu N razy

N = 6                                                # N razy fitujemy
repr_nr = '_rep3V3_'                                 # numer reprezentacji, będzie w nazwie pliku z modelem
n_epochs = 50                                        # liczba epok
model_out = 'drive/My Drive/models/rep3V3/50epoch/'  # gdzie zapiszemy model, foldery tworzymy w zależności od liczby epok

for k in range(0,N): 
  
  def make_model_modified(my_loss): # definiujemy za każdym razem, by initial weight się zmieniły? Czy się zmieniają przy wywołaniu compile?

      dense_layer_sizes=128
      filters=10
      kernel_size=(3,3) 
      pool_size= (2,2) 
      drop_out = 0.5

      ##  3 conv (3,3), na zmianę z (2,2) pooling
      model = Sequential()
      model.add(Conv2D(filters, kernel_size,input_shape=input_shape, activation='relu'))
      model.add(MaxPooling2D(pool_size=pool_size))
      model.add(Conv2D(filters, kernel_size, activation='relu'))
      model.add(MaxPooling2D(pool_size=pool_size))
      model.add(Conv2D(filters, kernel_size, activation='relu'))
      model.add(MaxPooling2D(pool_size=pool_size))

      ## Gęste warstwy: 128, 32, 1, dropout 0.5
      model.add(Flatten())
      model.add(Dense(dense_layer_sizes, activation='relu'))
      model.add(Dropout(drop_out))
      model.add(Dense(32, activation='relu'))
      model.add(Dropout(drop_out))
      model.add(Dense(1, activation='sigmoid'))

      model.compile(loss=my_loss, # 'binary_crossentropy',
                    optimizer='adam',#keras.optimizers.Adam(lr),#'adam',
                    metrics=['accuracy',auc_roc])

      return model

  ################################# Definicja modelu
  input_shape = (1, r, s)
  loss_custom = custom_loss2
  loss_bc = 'binary_crossentropy'

  my_model = make_model_modified(loss_bc)

  #################################

  now = datetime.datetime.now()
  start_time = str(now.year)+str('-')+str(now.month)+str('-')+str(now.day)+str('_')+str(now.hour+2)+str(':')+str(now.minute)
  
  y_frame = pd.DataFrame(y_train)
  print(np.shape(y_frame))

  print("Model binary crossentropy, bez wag")
  print("Run: " + str(k))
  grid_result = my_model.fit(X_train, y_train, batch_size = 32, epochs = n_epochs, validation_data=(X_val, y_val), verbose = 2)

  my_model.save((model_out + str(start_time) + '-' + str(n_epochs) + 'ep-' + str(repr_nr) + str(loss_bc) + '_run'+str(k+1)+'_my_model.h5')) 

  #########################################################


In [0]:
######## PREDYKCJA


del X_train, X_val  ## Wyrzucamy by nie zajmowało miejsca

'''
#################### Rep 1 - spektro ####################
# rep 1  ------- 63 x 148 ------ 

X_test1_14 = numpy.load('drive/My Drive/rep1/X_test2_rep1.npy', allow_pickle=True)   
y_test1_14 = numpy.load('drive/My Drive/rep1/y_test2_rep1.npy', allow_pickle=True)

X_test1_6 = numpy.load('drive/My Drive/rep1/X_test_rep1.npy', allow_pickle=True)  
y_test1_6 = numpy.load('drive/My Drive/rep1/y_test_rep1.npy', allow_pickle=True)  ### ----- 322 positive! -----

X_test1_6 = numpy.load('drive/My Drive/rep1/X_test1_rep1_nowa676.npy', allow_pickle=True)  
y_test1_6 = numpy.load('drive/My Drive/rep1/y_test1_rep1_nowa676.npy', allow_pickle=True)  ### ----- 676 positive! -----

# rep 1b ------- 63 x 63 ------
X_test1_14 = numpy.load('drive/My Drive/rep1b/X_test2_rep1b.npy', allow_pickle=True) 
y_test1_14 = numpy.load('drive/My Drive/rep1b/y_test2_rep1b.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep1b/X_test1_rep1b.npy', allow_pickle=True)  
y_test1_6 = numpy.load('drive/My Drive/rep1b/y_test1_rep1b.npy', allow_pickle=True)

#################### Rep 3 - mel-spektro ####################

# rep 3 ------- 60 x 111 ------ 
X_test1_14 = numpy.load('drive/My Drive/rep3/X_test2_rep3.npy', allow_pickle=True)  
y_test1_14 = numpy.load('drive/My Drive/rep3/y_test2_popr.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep3/X_test_rep3.npy', allow_pickle=True)   
y_test1_6 = numpy.load('drive/My Drive/rep3/y_test.npy', allow_pickle=True)

# rep 3b ------- 60 x 63 ------
X_test1_14 = numpy.load('drive/My Drive/rep3b/X_test2_rep3b.npy', allow_pickle=True)   
y_test1_14 = numpy.load('drive/My Drive/rep3b/y_test2_rep3b.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep3b/X_test1_rep3b.npy', allow_pickle=True)   
y_test1_6 = numpy.load('drive/My Drive/rep3b/y_test1_rep3b.npy', allow_pickle=True)

# rep 3V3 ------- 60 x 148 ------
X_test1_14 = numpy.load('drive/My Drive/rep3V3/X_test2_rep3V3.npy', allow_pickle=True)   
y_test1_14 = numpy.load('drive/My Drive/rep3V3/y_test2_rep3V3.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep3V3/X_test1_rep3V3.npy', allow_pickle=True)   
y_test1_6 = numpy.load('drive/My Drive/rep3V3/y_test1_rep3V3.npy', allow_pickle=True)

#################### Rep 5 - mel-spektro ####################

# rep 5 ------- 64 x 61 ------ 
X_test1_14 = numpy.load('drive/My Drive/rep5/X_test2_rep5.npy', allow_pickle=True)   
y_test1_14 = numpy.load('drive/My Drive/rep5/y_test2_rep5.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep5/X_test_rep5.npy', allow_pickle=True)   
y_test1_6 = numpy.load('drive/My Drive/rep5/y_test_rep5.npy', allow_pickle=True) ### ----- 322 positive! -----

X_test1_6 = numpy.load('drive/My Drive/rep5/X_test1_rep5_nowa676.npy', allow_pickle=True)  
y_test1_6 = numpy.load('drive/My Drive/rep5/y_test1_rep5_nowa676.npy', allow_pickle=True)  ### ----- 676 positive! -----

# rep 5b ------- 64 x 149 ------
X_test1_14 = numpy.load('drive/My Drive/rep5b/X_test2_rep5b.npy', allow_pickle=True)  
y_test1_14 = numpy.load('drive/My Drive/rep5b/y_test2_rep5b.npy', allow_pickle=True)
X_test1_6 = numpy.load('drive/My Drive/rep5b/X_test1_rep5b.npy', allow_pickle=True)   
y_test1_6 = numpy.load('drive/My Drive/rep5b/y_test1_rep5b.npy', allow_pickle=True)
'''



###

# model_out = 'drive/My Drive/models/rep3V2/'    # gdzie są zapisane modele
model_out_save = 'drive/My Drive/models/rep3V2/'  # gdzie zapiszemy predykcje

###
r = np.shape(X_test1_6)[1]
s = np.shape(X_test1_6)[2]

X_test1_14 = X_test1_14.reshape(X_test1_14.shape[0], 1, r, s).astype('float32')
y_test1_14 = np.squeeze(y_test1_14)
X_test1_6 = X_test1_6.reshape(X_test1_6.shape[0], 1, r, s).astype('float32')
y_test1_6 = np.squeeze(y_test1_6)

modele = []
modele += [each for each in sorted(os.listdir(model_out)) if each.endswith('.h5')] 
print(modele)

for k in range(0,6):  
  model = load_model(model_out + modele[k], custom_objects={'auc': auc_roc})
  print(modele[k])
  print(k)
  
  ### Predykcja na 20 nagraniach
  
  probs1 = model.predict_proba(X_test1_14)
  probs2 = model.predict_proba(X_test1_6)
  probs = np.concatenate([probs1, probs2])
  y_test1 = np.concatenate([y_test1_14, y_test1_6])
  
  ## Predykcja na 14 z 2018 
  
  #probs = model.predict_proba(X_test1_14)    
  #y_test1=y_test1_14
  
  ###
  
  print(np.shape(probs))
  print(np.shape(y_test1))
  auc = roc_auc_score(y_test1, probs)
  print("Run: " + str(k))
  print('AUC: %.6f' % auc)
  fpr, tpr, thresholds = roc_curve(y_test1, probs)

  print(np.shape(fpr))
  run = k +1
  np.savez((model_out_save + str(repr_nr)+ str(k+1)), fpr=fpr, tpr=tpr, thresholds=thresholds, run= run )

  a = np.load(model_out_save + str(repr_nr)+ str(k+1) + ".npz")
  print('Dostepne dane: ' + str(a.files))
  print(a["run"])